In [1]:
import os
import json
import base64
import openai
from openai import OpenAI
from dotenv import find_dotenv, load_dotenv
from uuid import uuid4
from random import randint
from utils.get_openai_api_key import get_test_key
from utils.get_postgres_connection import _conn_open
from utils.load_json import load_json
from rich.console import Console
import psycopg2

In [2]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="localhost",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [3]:
console = Console()

In [4]:
load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if OPENAI_API_KEY is None:
    OPENAI_API_KEY = get_test_key()
openai.api_key = OPENAI_API_KEY

In [5]:
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
console.print(f"[dark_orange bold]{OPENAI_API_KEY[:12]}[/]")

sk-proj-RRFX

In [6]:
def get_embedding(text, model="text-embedding-3-small"):

    text = text.replace("\n", " ")
    return client.embeddings.create(input=[text], model=model).data[0].embedding

In [7]:
def decode_image(encoded_image_data, image_fimename):
    decoded_image_data = base64.b64decode(encoded_image_data)
    file_name_path = f"./{image_fimename}"
    with open(file_name_path, "wb") as new_image_file:
        new_image_file.write(decoded_image_data)

In [8]:
FILENAME = "meetup.json"

In [9]:
def embed_and_load(filename=FILENAME):

    json_data = load_json(FILENAME)
    print(f"Number of items: {len(json_data)}")
    encoded_image_data = None
    file_id = str(uuid4())
    for i in range(len(json_data)):

        # if text_html then must be parsed for correct HTML quotes
        element_id = json_data[i]["element_id"]
        element_type = json_data[i]["type"].upper()
        if "parent_id" in json_data[i]:
            parent_id = json_data[i]["parent_id"]
        else:
            parent_id = ""
        if "page_number" in json_data[i]:
            page_number = json_data[i]["page_number"]
        else:
            page_number = ""

        filename = json_data[i]["metadata"]["filename"]
        element_text = json_data[i]["text"]
        if element_text is not None:
            element_embedding = get_embedding(element_text)
        else:
            element_embedding = ""
        if json_data[i]["type"] == "Image" or json_data[i]["type"] == "Table":
            console.print(
                f"[dark_orange]--------- {element_type.upper()} Item Number  {i + 1}--------[/]"
            )
            el_image_b64 = json_data[i]["metadata"]["image_base64"]
            # console.print(el_image_b64[:40])
        else:
            el_image_b64 = "NONE"

        sql = f"""
            INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('{file_id}','{element_id}','{element_text}','{element_type}','{parent_id}', 'page_number', '{el_image_b64}', '{element_embedding}')
            RETURNING id;
                """
        console.print(sql)
        try:
            # cur.execute(sql)
            # row = cur.fetchone()
            # id = row[0]
            # console.print(f"{i} - ID is {id}\nFILE_ID: {file_id}")
            conn.commit()
        except Exception as e:
            console.print(e)

In [10]:
files = ["meetup.json"]

In [11]:
for file in files:
    print(file)
    FILENAME = file
    embed_and_load(filename=FILENAME)
    print(f"file {file} loaded")

meetup.json
Number of items: 20


INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('8228c812-29ad-47ab-bb6f-e5c23bde6ad0','086853b9-7d2f-4770-988c-4f3f65150aee','AI Agents in the Data 
Pipeline','TITLE','', 'page_number', 'NONE', '[-0.038789600133895874, 0.005426667630672455, 0.035688646137714386, 
-0.003651233622804284, 0.019934697076678276, 0.0053436062298715115, -0.0070359790697693825, 0.020017758011817932, 
-0.0062192101031541824, 0.023035651072859764, 0.025721296668052673, -0.05390674248337746, -0.006918309256434441, 
-0.011566277593374252, -0.022149663418531418, -0.0011723748175427318, 0.03391667455434799, 0.005014822352677584, 
-0.04504687711596489, 0.02263418771326542, 0.018813369795680046, 0.0400908924639225, -0.0038000517524778843, 
0.013019849546253681, -0.013158285059034824, 0.0051290313713252544, -0.005021743942052126, 0.018135037273168564, 
-0.024904528632760048, -0.023769358173012733, 0.06395715475082397, -0.0312587134540081, -0.007177875377237797, 
0.040810756385326385, 0.04136449843645096, 0.0023326363880187273, -0.02990204654633999, 0.019173303619027138, 
-0.004689499270170927, 0.004450697917491198, 0.03801435977220535, 0.007870052009820938, -0.03538408875465393, 
0.03175707906484604, -0.03416585549712181, 0.03602088987827301, 0.0037585210520774126, -0.046957287937402725, 
-0.01812119409441948, 0.010528012178838253, -0.049393750727176666, 0.023492487147450447, -0.04313647001981735, 
-0.023077180609107018, 0.009088284336030483, 0.018204255029559135, 0.009420529007911682, 0.009600494988262653, 
0.03372286260128021, -0.043690212070941925, -0.03136946260929108, -0.01741517335176468, -0.03602088987827301, 
0.04346871376037598, -0.022274255752563477, -0.012002348899841309, -0.03164633363485336, 0.003921182826161385, 
-0.05423898994922638, -0.01068029087036848, 0.03067728504538536, 0.03599320352077484, -0.0012727404246106744, 
-0.010479560121893883, -0.03685150295495987, -0.013774322345852852, -0.029735922813415527, 0.01503408420830965, 
-0.020031603053212166, -0.03516259044408798, -0.0015885462053120136, 0.0034660762175917625, 0.008313045836985111, 
-0.07768994569778442, -0.023672454059123993, -0.050196677446365356, -0.023644765838980675, -0.018965650349855423, 
-0.05836436524987221, -0.010140392929315567, -0.016058506444096565, 0.0568692609667778, -0.043496403843164444, 
-0.016404595226049423, 0.023007962852716446, 0.04133680835366249, 0.02350633032619953, -0.01690296269953251, 
0.03602088987827301, 0.017982758581638336, 0.00286907353438437, -0.043662525713443756, -0.023478643968701363, 
0.03361211344599724, 0.011683947406709194, -0.03485803306102753, -0.016210785135626793, 0.007080970797687769, 
-0.057035382837057114, -0.002775629749521613, -0.04568368196487427, -0.044908441603183746, -0.029514426365494728, 
-0.018356533721089363, -0.01607234962284565, 0.033473677933216095, -0.001699294545687735, 0.0288776233792305, 
0.02187279239296913, -0.014494186267256737, -0.027451738715171814, 0.03519027680158615, 0.011483216658234596, 
-0.005447432864457369, -0.012334593571722507, 0.023035651072859764, 0.00837534200400114, -0.09624028950929642, 
0.012736056931316853, -0.014002740383148193, 0.01629384607076645, -0.026787249371409416, 0.021817419677972794, 
0.026012010872364044, -0.0061396099627017975, -0.009032909758388996, -0.012563012540340424, -0.009032909758388996, 
-0.006634516641497612, 0.004845238756388426, 0.023478643968701363, 0.009593573398888111, 0.001304753590375185, 
0.03228313475847244, -0.04111531376838684, 0.010534933768212795, -0.050888851284980774, 0.037349872291088104, 
0.033058371394872665, 0.022938745096325874, -0.008617603220045567, 0.02538905292749405, -0.03233850747346878, 
-0.02382473275065422, -0.014563404023647308, 0.019574765115976334, -0.013891992159187794, 0.030400414019823074, 
0.0339997336268425, -0.05108266323804855, -0.033030685037374496, 0.03679613023996353, 0.02221888117492199, 
-2.7605965442489

INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('8228c812-29ad-47ab-bb6f-e5c23bde6ad0','d30015c7-1598-456e-a3e7-beeb3cd912b0','Can we build an Agentic
AI Data Analyst (intern)?','UNCATEGORIZEDTEXT','', 'page_number', 'NONE', '[-0.027628574520349503, 
0.02526753395795822, 0.009957431815564632, -0.008982219733297825, 0.010140742175281048, -0.01076399814337492, 
0.01670326106250286, 0.029549669474363327, -0.004553435370326042, -0.0015049800276756287, -0.0038421901408582926, 
-0.051972221583127975, 0.002672668546438217, -0.03780597820878029, -0.0044251177459955215, -0.0006415871321223676, 
0.008674257434904575, -0.04431717097759247, -0.029344361275434494, 0.022979816421866417, 0.023302443325519562, 
0.04443448781967163, -0.008586268872022629, -0.0006081329192966223, 0.042058784514665604, -0.030561544001102448, 
0.006643176078796387, 0.025590158998966217, -0.0045387702994048595, -0.015500744804739952, 0.020061511546373367, 
-0.028728436678647995, -0.02248121239244938, 0.00045621427125297487, 0.02526753395795822, 0.03730737417936325, 
0.004469112493097782, 0.010162740014493465, -0.022363893687725067, 0.004553435370326042, -0.009480824694037437, 
-0.01981220953166485, 0.02641139179468155, 0.04443448781967163, -0.008718252182006836, -0.005396664142608643, 
-0.016351304948329926, -0.008124325424432755, 0.018814999610185623, 0.014173575676977634, -0.06751696020364761, 
-0.02253987081348896, 0.026294073089957237, 0.02085341326892376, -0.043554600328207016, -0.010316720232367516, 
-0.013960935175418854, 0.0026855003088712692, -0.015867365524172783, -0.017113877460360527, 0.007603723555803299, 
-0.004601096268743277, 0.025531500577926636, 0.04906858131289482, -0.00012579689791891724, -0.004901725333184004, 
-0.01771513558924198, 0.022686520591378212, -0.031646743416786194, -0.007255433592945337, 0.012267145328223705, 
0.020706765353679657, 0.004513106774538755, -0.023111799731850624, -0.0024691938888281584, -0.015735382214188576, 
-0.04898059368133545, 0.019357599318027496, -0.026426056399941444, -0.011907856911420822, -0.03276127204298973, 
0.041530851274728775, -0.007655050605535507, -0.030473554506897926, -0.0038715198170393705, -0.00208057533018291, 
-0.0476020984351635, -0.01661527343094349, -0.0656985193490982, -0.023331772536039352, 0.008769579231739044, 
0.039419110864400864, -0.03217467665672302, -0.03739536181092262, 0.08916227519512177, 0.023757053539156914, 
0.03519563376903534, 0.011005968786776066, 0.012736420147120953, 0.03331853449344635, -0.021777300164103508, 
-0.03927246481180191, 0.00827830657362938, 0.019782880321145058, -0.00019155957852490246, -0.008813573978841305, 
0.02214392088353634, 0.029285700991749763, -0.010412042029201984, -0.011401919648051262, -0.07150579988956451, 
-0.035723570734262466, -0.0065918490290641785, -0.03874452784657478, -0.04815936088562012, -0.03780597820878029, 
-0.051238980144262314, -0.016600608825683594, -0.029740311205387115, -0.057838160544633865, 0.008938224986195564, 
0.0234930869191885, 0.013535655103623867, -0.009715462103486061, -0.00818298477679491, -0.029652323573827744, 
-0.022613195702433586, -0.07297228276729584, -0.047455448657274246, -0.03724871575832367, 0.010756665840744972, 
0.006610180251300335, 0.013447665609419346, -0.04305599257349968, -0.01456952653825283, -0.06364543735980988, 
-0.010243396274745464, 0.0019265944138169289, -0.025956781581044197, -0.035723570734262466, 0.021821293979883194, 
-0.007453409023582935, -0.009451494552195072, 0.015882030129432678, -0.03323054686188698, 0.0028339819982647896, 
0.030532212927937508, 0.019944192841649055, 0.020560117438435555, 0.02773122861981392, -0.008754914626479149, 
0.015280772000551224, -0.03390512987971306, -0.04566633701324463, -0.0011915188515558839, 0.013843616470694542, 
0.011211276054382324, 0.011343260295689106, -0.017201866954565048, -0.07015663385391235, 0.016233986243605614, 
0.033

INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('8228c812-29ad-47ab-bb6f-e5c23bde6ad0','ba736bd2-f125-4e35-bbbc-f0d3552b99db','HANDS ON 
WORKSHOP','UNCATEGORIZEDTEXT','', 'page_number', 'NONE', '[-0.005078763701021671, 0.03195495158433914, 
0.011530363000929356, -0.0038884852547198534, -0.011048410087823868, -0.016620080918073654, -0.0029556138906627893,
0.025207608938217163, 0.005670251324772835, -0.02294388972222805, 0.0668308287858963, -0.034379322081804276, 
0.008521807380020618, -0.06256628036499023, 0.01063948031514883, 0.0173795223236084, 0.02292928472161293, 
0.0033462881110608578, 0.028712721541523933, 0.022578774020075798, 0.031078673899173737, 0.00953682977706194, 
-0.020694775506854057, 0.017978312447667122, 0.011829758062958717, -0.01565617322921753, 0.011793246492743492, 
-0.01498436089605093, 0.0173064973205328, -0.0072256457060575485, 0.027602769434452057, -0.03870229423046112, 
0.007291366346180439, -0.009325062856078148, -0.002577719045802951, 0.06128107011318207, 0.032568346709012985, 
0.025879422202706337, -0.021483425050973892, -0.010113713331520557, 0.01228250190615654, -0.029194675385951996, 
0.013582315295934677, 0.04092220216989517, 0.018197381868958473, 0.02637597918510437, -0.010113713331520557, 
0.028712721541523933, 0.006261739414185286, 0.019687054678797722, 0.034262485802173615, 0.00975589919835329, 
-0.007638226728886366, 0.020388077944517136, -0.03680369257926941, -0.022345099598169327, -0.0010332782985642552, 
-0.00041554763447493315, -0.021483425050973892, 0.018299613147974014, 0.039549365639686584, -0.010427712462842464, 
-0.00032358404132537544, 0.007284064311534166, -0.03727104142308235, -0.004111206158995628, -0.0665387362241745, 
0.011654502712190151, -0.04784480109810829, 0.025864817202091217, 0.04384312778711319, 0.051700424402952194, 
-0.03157523274421692, 0.023221377283334732, -0.0016868358943611383, 0.025382863357663155, -0.021892355754971504, 
0.02820155955851078, -0.0055388095788657665, 0.038030482828617096, -0.013253710232675076, -0.009887341409921646, 
0.04731903225183487, -0.00970478355884552, -0.09615694731473923, -0.06478618085384369, -0.09113295376300812, 
-0.027033187448978424, -0.05120386555790901, -0.0016229406464844942, 0.006455251015722752, 0.04349261894822121, 
0.028581280261278152, 0.031078673899173737, 0.053365353494882584, -0.008149389177560806, 0.010332782752811909, 
0.008090970106422901, -0.005239414516836405, 0.03887755051255226, 0.07594412565231323, -0.006247134879231453, 
-0.019906124100089073, -0.007857296615839005, -0.0030925325118005276, -0.002296579536050558, -0.03741708770394325, 
-0.012187572196125984, -0.05608181655406952, -0.005170042626559734, -0.14207392930984497, -0.07769668102264404, 
0.009726690128445625, 0.02801170013844967, 0.02972044236958027, 0.005568019114434719, 0.04378471150994301, 
-0.042791593819856644, 0.005608181469142437, -0.045069918036460876, -0.02459421381354332, -0.013041943311691284, 
-0.020022960379719734, -0.024141469970345497, -0.027748815715312958, -0.030465278774499893, -0.0320717878639698, 
-0.012917803600430489, 0.003234927775338292, -0.05757148936390877, 0.0025065212976187468, -0.043025270104408264, 
0.04705614969134331, -0.0457417331635952, 0.021775517612695694, 0.0060682278126478195, -0.01959942653775215, 
0.06379306316375732, -0.022228261455893517, -0.01488212775439024, 0.0320717878639698, -0.014276035130023956, 
-0.0013591443421319127, 0.0022144285030663013, -0.025791794061660767, -0.029048629105091095, 0.0173064973205328, 
0.026682676747441292, 0.019000636413693428, 0.030319232493638992, -0.03651160001754761, -0.03204258158802986, 
-0.025528909638524055, -0.02672649174928665, 0.009441900067031384, -0.04407680407166481, -0.025149188935756683, 
0.02656584046781063, -0.05961614102125168, -0.008967248722910881, -0.0035525786224752665, -0.01607970893383026, 
-0.03137076646089554, 0.03271439298987

INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('8228c812-29ad-47ab-bb6f-e5c23bde6ad0','ee6ae524-4de5-4976-8e48-40c716f0890a','In this 90 minute 
workshop we will work almost exclusively with notebooks.','NARRATIVETEXT','', 'page_number', 'NONE', 
'[-0.058085303753614426, 0.05379781126976013, 0.03718378022313118, -0.06293424963951111, 0.016039812937378883, 
-0.041062939912080765, 0.027154114097356796, 0.033712953329086304, -0.0017003221437335014, 0.0063833859749138355, 
-6.5733461269701365e-06, 0.00614731851965189, 0.015363511629402637, -0.007892302237451077, -0.021781988441944122, 
0.021718187257647514, 0.02109292708337307, 0.003509107744321227, 0.026643697172403336, 0.004612881690263748, 
0.0014993459917604923, -0.028072861954569817, 0.0017832648009061813, 0.007375506218522787, -0.013934347778558731, 
-0.02060803212225437, -0.015057262033224106, -0.005349794402718544, 0.03509107604622841, -0.028200466185808182, 
-0.002453185385093093, -0.03327910229563713, 0.017392413690686226, 0.002105464693158865, -0.02651609480381012, 
-0.005381694994866848, 0.03560149297118187, 0.02544422075152397, -0.04369158297777176, 0.018808817490935326, 
-0.026796823367476463, 0.015172106213867664, -0.0022553992457687855, -0.012734871357679367, -0.0009817526442930102,
0.04461032897233963, 0.01426611840724945, 0.028302548453211784, 0.005991003941744566, 0.013832264579832554, 
-0.020467668771743774, 0.00833253562450409, 0.010916515253484249, -0.0335087887942791, 0.025699429214000702, 
0.019804127514362335, -0.08039046823978424, 0.050505630671978, 0.03187545761466026, 0.01884709857404232, 
0.06176029518246651, -0.006839570123702288, 0.07584776729345322, 0.009940344840288162, 0.05267489701509476, 
0.012345679104328156, -0.06905923783779144, 0.004695824347436428, 0.009213002398610115, -0.0019890260882675648, 
0.01575908437371254, -0.010520943440496922, -0.062423836439847946, -0.00032718441798351705, 0.01364085916429758, 
-0.01237757969647646, -0.026120521128177643, -0.0010670877527445555, 0.004188598599284887, 0.02351740188896656, 
-0.016129136085510254, 0.01942131668329239, -0.01801767386496067, 0.015159345231950283, -0.057983219623565674, 
-0.023810891434550285, -0.03736242651939392, -0.01877053640782833, -0.023083548992872238, -0.02776661328971386, 
0.013207005336880684, 0.023874692618846893, -0.000149037383380346, 0.0017944300780072808, 0.0450441837310791, 
0.02451271191239357, -0.016448145732283592, -0.0022346635814756155, 0.020556991919875145, 0.041828565299510956, 
0.07145819067955017, 0.03292180970311165, 0.03427441045641899, -0.020161418244242668, 0.033483266830444336, 
-0.012505183927714825, -0.028149424120783806, 0.0028455674182623625, -0.05175614729523659, -0.049995213747024536, 
-0.06717070192098618, -0.052317604422569275, 0.023976776748895645, -0.0662519559264183, -0.00642166705802083, 
-0.032156188040971756, 0.023798130452632904, -0.02950202487409115, 0.038817111402750015, -0.05111812800168991, 
0.02368328720331192, -0.0019363894825801253, -0.024066098034381866, 0.008708967827260494, -0.04126710817217827, 
-0.03327910229563713, -0.019638242200016975, 0.040884293615818024, 0.009155580773949623, -0.02769004926085472, 
0.0285322368144989, -0.04726449027657509, 0.01777522638440132, -0.015274188481271267, -0.004625642206519842, 
0.02225412242114544, -0.039072319865226746, 0.046320222318172455, -0.0034229750744998455, 0.00278176530264318, 
0.015657000243663788, 0.002237853594124317, 0.07161131501197815, 0.005892110988497734, -0.00044222731958143413, 
0.03695409372448921, 0.02092704176902771, 0.07870609313249588, -0.014495804905891418, -0.002931700088083744, 
-0.004657542798668146, -0.054410308599472046, -0.045554596930742264, -0.0038121670950204134, 0.011688519269227982, 
-0.00311832083389163, -0.009806361049413681, 0.03292180970311165, -0.0027275336906313896, 0.004781956784427166, 
0.013513254933059216, -0.0127348713

...

INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('8228c812-29ad-47ab-bb6f-e5c23bde6ad0','6f2957ee-5269-451c-85a0-56c7e8f85551','Groq provides a free 
tier which will be more than sufficient for our use. It uses the same interface as OpenAI so an OpenAI API key or 
Groq API key will be sufficient.','NARRATIVETEXT','', 'page_number', 'NONE', '[-0.02292908914387226, 
0.0096972044557333, 0.04832053929567337, -0.04471966624259949, 0.01901049166917801, -0.05454263836145401, 
-0.021764101460576057, 0.05803760141134262, 0.006688754074275494, -0.053721848875284195, 0.034128863364458084, 
-0.04363410919904709, -0.039424266666173935, -0.016309836879372597, 0.015833251178264618, -0.02667558565735817, 
-0.0210624597966671, -0.03489669784903526, -0.04524920880794525, 0.07111724466085434, 0.006304837763309479, 
0.011947750113904476, -0.010273079387843609, 0.003733258228749037, -0.012960495427250862, -0.005709105171263218, 
-0.005166326183825731, 0.007148792501538992, 0.010994577780365944, -0.03627350181341171, 0.02273051254451275, 
-0.03179888799786568, -0.06312118470668793, -0.00430582370609045, 0.02809210680425167, -0.026079853996634483, 
0.02081092819571495, 0.05245095491409302, 0.0360352098941803, 0.0008687768713571131, -0.011007816530764103, 
-0.050676994025707245, 0.005123300943523645, 0.01444320846349001, 0.010915147140622139, 0.023101190105080605, 
-0.006652348209172487, 0.007214984856545925, -0.016759946942329407, -0.014244630932807922, -0.04609647020697594, 
-0.04342229291796684, -0.03979494422674179, -0.0077710021287202835, -0.015687627717852592, 0.011067389510571957, 
-0.03574396297335625, 0.013139215297996998, 0.02780085988342762, 0.0109681012108922, -0.03614111617207527, 
0.019791563972830772, 0.021247798576951027, -0.0016506760148331523, 0.034287724643945694, 0.014284346252679825, 
-0.0112394904717803, 0.005298711359500885, -0.034711357206106186, 0.04167481139302254, -0.03537328168749809, 
-0.008161538280546665, -0.0013453629799187183, -0.04577874764800072, 0.0029058516956865788, -0.008591788820922375, 
-0.04710259661078453, 0.06412731111049652, 0.010392226278781891, -0.030898669734597206, -0.038497570902109146, 
0.0036141115706413984, -0.019341453909873962, -0.023418914526700974, 0.004554045386612415, -0.034578971564769745, 
-0.040192097425460815, -0.015912681818008423, -0.0158067736774683, 0.01686585508286953, -0.003776283236220479, 
-0.013675374910235405, -0.03209013491868973, -0.03566453233361244, 0.017488064244389534, 0.024491233751177788, 
0.019765086472034454, 0.017554257065057755, 0.003380783135071397, 0.046705443412065506, -0.015793535858392715, 
-0.02462361752986908, -0.04802929237484932, -0.04005971550941467, 0.037756215780973434, 0.02340567484498024, 
0.07254700362682343, 0.032752059400081635, -0.02721836417913437, -0.004941271618008614, -0.043078094720840454, 
-0.03190479427576065, -0.00028752375510521233, -0.015634672716259956, -0.009061756543815136, -0.018123513087630272,
-0.050068024545907974, -0.00021884901798330247, -0.010729807429015636, -0.030633898451924324, 
-0.022809943184256554, -3.956037471652962e-05, 0.008830082602798939, -0.004871769342571497, 0.01442996971309185, 
-0.057349201291799545, 0.010173790156841278, -0.029283571988344193, -0.05226561427116394, -0.00934638362377882, 
0.00878374744206667, -0.02488838881254196, 0.035505667328834534, 0.047182030975818634, -0.022783465683460236, 
-0.0183088518679142, -0.04313104599714279, -0.02141989953815937, -0.014151960611343384, -0.03802098333835602, 
-0.0030878810212016106, -0.09537018835544586, 0.009022040292620659, 0.03399647772312164, -0.049591436982154846, 
-0.026781493797898293, -0.04164833575487137, -0.027019787579774857, -0.017845503985881805, 0.022783465683460236, 
-0.007459897082298994, -0.03463192656636238, -0.05184198170900345, -0.0023647279012948275, -0.0094258151948452, 
0.015674389898777008, -0.0011881557293236256, 0

INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('8228c812-29ad-47ab-bb6f-e5c23bde6ad0','8ba18d2c-2f35-4d7a-964e-07e312df611a','Free Tier with Groq: 
https://console.groq.com/login','UNCATEGORIZEDTEXT','', 'page_number', 'NONE', '[-0.03507669270038605, 
-0.001281312550418079, 0.023323904722929, -0.021730780601501465, 0.024637535214424133, -0.010697692632675171, 
-0.054166242480278015, 0.07708487659692764, -0.0096565717831254, -0.01615484431385994, 0.0628306046128273, 
-0.03367921710014343, 0.006962933577597141, -0.028550472110509872, 0.012968594208359718, -0.007749014534056187, 
-0.0345456525683403, -0.005188136361539364, -0.03778780251741409, 0.10570522397756577, 0.00756734237074852, 
0.0242741908878088, -0.013758169487118721, 0.01915942132472992, -0.007504456210881472, 0.003622961463406682, 
-0.02336582913994789, 0.015875348821282387, 0.02124166302382946, -0.04885582625865936, 0.031051957979798317, 
-0.04248332604765892, -0.022024251520633698, -0.0013468193355947733, 0.022848762571811676, -0.006547183729708195, 
0.026803625747561455, 0.06366909295320511, 0.050225354731082916, 0.01560982782393694, 0.0027233350556343794, 
-0.012060234323143959, 0.024679459631443024, 0.011389444582164288, 0.07786746323108673, 0.005244035739451647, 
0.014980962499976158, 0.006617057602852583, -0.030968109145760536, -0.008287043310701847, -0.04656396061182022, 
-0.03748035430908203, -0.0446074940264225, -0.003572302870452404, -0.05852637439966202, 0.011005138047039509, 
-0.0036963289603590965, 0.016853582113981247, 0.005513050127774477, -0.003850051434710622, -0.029095489531755447, 
0.008552564308047295, -0.0012673377059400082, 0.004265801049768925, 0.019620589911937714, 0.015316356904804707, 
-0.02528037503361702, -0.013779131695628166, -0.0076302289962768555, 0.039017580449581146, -0.0320860929787159, 
-0.009852218441665173, 0.021632956340909004, -0.027628136798739433, 0.012751985341310501, 0.011179822497069836, 
-0.008384867571294308, 0.05640220642089844, 0.015414181165397167, -0.02934703417122364, -0.02009573206305504, 
-0.035719532519578934, -0.042092032730579376, -0.007749014534056187, 0.024413937702775, -0.019606614485383034, 
-0.08832059800624847, -0.014533769339323044, -0.02437201328575611, 0.02023547887802124, 0.008629425428807735, 
-0.01055095810443163, -0.013401812873780727, -0.013730219565331936, 0.01162701565772295, -0.0028927791863679886, 
0.02624463476240635, 0.009286240674555302, -0.03952067345380783, 0.021521158516407013, 0.008992769755423069, 
-0.06193621829152107, -0.07132726907730103, -0.05534012243151665, 0.038766033947467804, 0.013206166215240955, 
0.06193621829152107, 0.03622262552380562, 0.012968594208359718, 0.004534815438091755, -0.06053873896598816, 
-0.05162283033132553, -0.010068828240036964, -0.0059777116402983665, 0.022667091339826584, 0.010516020469367504, 
-0.008894946426153183, -0.020682672038674355, -0.004664082545787096, -0.0323655866086483, -0.02013765648007393, 
0.015316356904804707, -0.008321980945765972, -0.019900085404515266, 0.009950042702257633, -0.011026100255548954, 
-0.0365859717130661, -0.03317612409591675, -0.06780562549829483, -0.0034168334677815437, 0.016993330791592598, 
-0.023505577817559242, 0.050057657063007355, -0.01718897745013237, -0.01116584800183773, 0.02221989817917347, 
-0.021437309682369232, -0.037871651351451874, -0.0025259412359446287, 0.0046501075848937035, -0.03775985166430473, 
-0.05542397126555443, 0.04642421379685402, 0.015372256748378277, -0.038933731615543365, -0.03734060749411583, 
-0.03353947028517723, 0.028871892020106316, -0.03549593687057495, 0.03966042026877403, 0.01623869314789772, 
-0.02226182259619236, -0.0482688844203949, 0.03353947028517723, 0.010823465883731842, -0.007462531793862581, 
0.010453134775161743, 0.053160056471824646, 0.017943615093827248, -0.0038989633321762085, -0.01116584800183773, 
-0.014994936995208263, -9.099982707994059

INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('8228c812-29ad-47ab-bb6f-e5c23bde6ad0','d4968bdb-75ff-4ff0-93f1-ec8dca66ffd4','GitHub Repo: TO 
FOLLOW...','UNCATEGORIZEDTEXT','', 'page_number', 'NONE', '[-0.022713536396622658, 0.004481867421418428, 
0.006364522036164999, -0.009382854215800762, 0.007503578905016184, -0.0513758584856987, -0.00022392437676899135, 
0.012965643778443336, -0.02804040163755417, -0.023768093436956406, 0.020726101472973824, -0.016507873311638832, 
-0.039910927414894104, -0.06624781340360641, 0.02906791865825653, 0.013553761877119541, 0.005164625588804483, 
-0.02860824018716812, 0.026323366910219193, 0.04207412153482437, -0.028932718560099602, -0.026120567694306374, 
0.01592651568353176, 0.04942898079752922, -0.03344838693737984, -0.031771913170814514, -0.053755369037389755, 
-0.007050660438835621, -0.03258311003446579, -0.01846826821565628, -0.01887386664748192, -0.04837442561984062, 
0.02751312404870987, 0.04142516478896141, 0.027851121500134468, 0.04759026691317558, 0.04869890585541725, 
-0.05305233225226402, 0.0022967036347836256, 0.04840146377682686, 0.0073886592872440815, -0.022483697161078453, 
0.023321935907006264, 0.013567281886935234, 0.006928980816155672, 0.006803920958191156, -0.029797997325658798, 
-0.0323667898774147, 0.026363926008343697, 0.03425958380103111, 0.006543661933392286, -0.0463193915784359, 
-0.0090718949213624, 0.04448067769408226, -0.027932241559028625, -0.016223954036831856, -0.0495641827583313, 
-0.02003658376634121, 0.005925123579800129, 0.0017373152077198029, -0.007442739326506853, -0.008429696783423424, 
0.0019367346540093422, -0.03063623420894146, -0.02920311875641346, -0.006928980816155672, -0.02686416544020176, 
0.007936217822134495, 0.03544934093952179, 0.012560045346617699, -0.007131780032068491, 0.028013361617922783, 
0.028797520324587822, -0.038667093962430954, 0.017575951293110847, -0.03650389984250069, 0.034043263643980026, 
0.045562274754047394, 0.021753618493676186, -0.0506187379360199, -0.0014880408998578787, 0.012188246473670006, 
-0.041911881417036057, -0.035800859332084656, -0.028310801833868027, -0.0599745512008667, -0.087663434445858, 
-0.08636552095413208, -0.035936061292886734, 0.02509305067360401, -0.025917768478393555, -0.023714013397693634, 
-0.059866394847631454, 0.05202481523156166, 0.04518371447920799, -0.010870049707591534, -0.014831398613750935, 
0.009355814196169376, 0.028959758579730988, 0.05267377197742462, 0.0101467315107584, -0.05513440817594528, 
-0.02686416544020176, -0.01990138366818428, 0.039180852472782135, -0.02160489931702614, 0.013567281886935234, 
-0.07111500203609467, -0.03874821215867996, -0.006036662962287664, -0.05435024946928024, -0.01821138896048069, 
0.006459162104874849, 0.010951169766485691, 0.05332273244857788, 0.024687450379133224, 0.03155559301376343, 
-0.011748847551643848, 0.004241888411343098, -0.01362812239676714, -0.02445761114358902, 0.03585493937134743, 
0.014574519358575344, -0.038207411766052246, 0.036206457763910294, -0.024471132084727287, 0.027797043323516846, 
-0.020969461649656296, -0.02082074247300625, -0.008044377900660038, 0.030528074130415916, -0.03542230278253555, 
0.0114108482375741, -0.010836250148713589, 0.012911563739180565, 0.015547956340014935, -0.031582631170749664, 
-0.004934786353260279, -0.020604422315955162, -0.021996978670358658, -0.025850167497992516, -0.03785589337348938, 
0.023822173476219177, -0.01612931489944458, 0.0015125457430258393, -0.017535390332341194, 0.0017626651097089052, 
-0.011620407924056053, 0.041911881417036057, 0.01600763574242592, -0.02933831885457039, 0.011383808217942715, 
-0.03763957694172859, 0.017251471057534218, -0.012039526365697384, -0.019360585138201714, 0.011620407924056053, 
0.014750278554856777, -0.05683792009949684, -0.0025282329879701138, -0.0013384763151407242, 0.023389535024762154, 
0.03077143430709839, 0.04045172780752182, 0

INSERT INTO tbl_doc_elements (
                file_id, element_id, element_text, element_type, parent_id, page_number, image_base64, embedding)
            VALUES 
            ('8228c812-29ad-47ab-bb6f-e5c23bde6ad0','bfbcdacc-9659-482e-b3ba-e85569bdc339','Craig West LinkedIn: 
https://www.linkedin.com/in/craig-west-0914b230b/ Website: https://craig-west.netlify.app/','UNCATEGORIZEDTEXT','',
'page_number', 'NONE', '[-0.0644313246011734, -0.03234889730811119, 0.042128175497055054, 0.026273485273122787, 
-0.03770485147833824, -0.022596264258027077, -0.019665144383907318, 0.04063596948981285, -0.03083004429936409, 
0.0321623720228672, 0.03354799002408981, 0.016147801652550697, -0.010758539661765099, 0.007840743288397789, 
-0.01991828717291355, 0.0628858283162117, -0.03194919973611832, -0.0134298549965024, -0.015534930862486362, 
0.03746503219008446, -0.007847405038774014, -0.0337611623108387, -0.01673402637243271, 0.053905945271253586, 
0.01334991492331028, 0.030936630442738533, 0.012623797170817852, 0.004829684738069773, -0.011058312840759754, 
0.008167163468897343, 0.050095491111278534, -0.030643517151474953, -0.01826620101928711, 0.04279433935880661, 
0.003927033394575119, 0.018212908878922462, 0.021530402824282646, 0.01610783115029335, 0.03954346105456352, 
0.03024381957948208, -0.013576410710811615, -0.002981081372126937, 0.04050273820757866, 0.05457210913300514, 
0.04068926349282265, -0.02532753348350525, -0.02286272868514061, 0.021383846178650856, 0.005472532473504543, 
0.03354799002408981, -0.015601547434926033, -0.024754634127020836, 0.008480260148644447, 0.04998890310525894, 
-0.04924280196428299, 0.018785808235406876, 0.01213083602488041, -0.03354799002408981, 0.010711908340454102, 
0.014309190213680267, 0.05878226086497307, -0.039943158626556396, -0.013376561924815178, -0.0022399744484573603, 
-0.04801706224679947, -0.042234763503074646, -0.020744329318404198, 0.005099480971693993, -0.01931873895227909, 
0.015028647147119045, 0.048709869384765625, 0.026633214205503464, 0.030110586434602737, 0.043007511645555496, 
-0.01903895102441311, -0.047031138092279434, -0.022023363038897514, 0.06357863545417786, -0.04591198265552521, 
-0.045618873089551926, -0.045778751373291016, 0.003550651017576456, -0.0191855076700449, -0.029524363577365875, 
-0.05457210913300514, -0.0021683620288968086, 0.0076009249314665794, -0.024594753980636597, -0.0027362662367522717,
0.0660301223397255, 0.01596127636730671, -0.01097837369889021, 0.0012207444524392486, 0.014668919146060944, 
-0.015708133578300476, 0.03019052743911743, 0.012843631207942963, 0.09534130990505219, 0.021090734750032425, 
-0.0383710153400898, -0.009706000797450542, -0.043167389929294586, 0.01991828717291355, -0.00834702793508768, 
0.062033139169216156, -0.022303150966763496, -0.0029561002738773823, 0.031203094869852066, -0.01610783115029335, 
0.014335837215185165, -0.12257406860589981, 0.05204068869352341, -0.058355916291475296, 0.010711908340454102, 
-0.0353066623210907, -0.026286808773875237, -0.012190790846943855, -0.05196074768900871, -0.02808544971048832, 
0.0034407342318445444, -0.038131196051836014, 0.03938358277082443, -0.0268597099930048, -0.05307990312576294, 
-0.006338545121252537, -0.007787450682371855, -0.02502109855413437, -0.0003949100209865719, -0.031496208161115646, 
-0.06608341634273529, 0.024621400982141495, -0.043966785073280334, 0.06773550063371658, 0.01019230019301176, 
-0.03312164545059204, -0.011018343269824982, -0.02773904614150524, -0.0475907176733017, -0.025540705770254135, 
-0.005256029311567545, 0.03482702374458313, 0.027072882279753685, 0.0214904323220253, 0.001910223625600338, 
-0.014575655572116375, 0.034134212881326675, -0.0007356942514888942, -0.02547409012913704, 0.009079807437956333, 
-0.0021583696361631155, -0.0506284199655056, -0.007507661823183298, -0.011358086951076984, -0.03141626715660095, 
-0.020184751600027084, -0.029817475005984306, 0.039516814053058624, 0.02254297025501728, -0.006092064548283815, 
-0.0207176823168993, 0.026260161772

file meetup.json loaded


In [12]:
cur.close()
conn.close()